In [1]:
import types
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3
def __iter__(self): return 0
client_bb25f14d4df64666b23e2f24f875ea53 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='SaoXtYUJrtE9Oh-kP6ZyRPbYO8-fzm3j4pFxKFovltTW',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')
body = client_bb25f14d4df64666b23e2f24f875ea53.get_object(Bucket='capstoneproject-donotdelete-pr-tsoy8yandjxtj3',Key='US Population data.csv')['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
df = pd.read_csv(body)

body = client_bb25f14d4df64666b23e2f24f875ea53.get_object(Bucket='capstoneproject-donotdelete-pr-tsoy8yandjxtj3',Key='US States.csv')['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
statesdf = pd.read_csv(body)
statesdf.rename(columns={'metropolitan':'States'},inplace=True)
statesdf['States'] = statesdf['States'].shift(5)
statesdf.rename(columns={'States':'State'},inplace=True)

In [2]:
firstdf=df.drop(df.iloc[:,12:19],axis=1)
firstdf.drop(['Unnamed: 1','Unnamed: 2','Unnamed: 4','Unnamed: 5'],inplace=True,axis=1)
firstdf.rename(columns = {'Cumulative Estimates of the Components of Population Change':'Population Change 2010-2015','Unnamed: 6':'Natural Increase','Unnamed: 7':'Event:Births','Unnamed: 8':'Event:Deaths','Unnamed: 9':'Net Migration','Unnamed: 10':'International Migration','Unnamed: 11':'Domestic Migration'}, inplace = True)
firstdf.drop(index=firstdf.index[[0, 1,2]],inplace=True)
firstdf.fillna('', inplace=True)
firstdf = firstdf.replace(',','', regex=True)
firstdf.drop(firstdf.index[0],inplace=True) 
firstdf['Geography'] = firstdf['Geography'].str[4:]
firstdf['City'] = firstdf['Geography'].apply(lambda x: x.split(" ", 1)[0])
cols=['Geography', 'City']
new_cols= cols+(firstdf.columns.drop(cols).tolist())
firstdf = firstdf[new_cols]
firstdf.insert(2, 'State', statesdf['State'])

In [3]:
seconddf=df.drop(df.iloc[:,1:12],axis=1)
seconddf.rename(columns={'Annual Estimates of the Components of Population Change':'Population Change 2014-2015','Unnamed: 13':'Natural Increase','Unnamed: 14':'Event:Births','Unnamed: 15':'Event:Deaths','Unnamed: 16':'Net Migration','Unnamed: 17':'International Migration','Unnamed: 18':'Domestic Migration'},inplace=True)
seconddf.drop(index=seconddf.index[[0, 1,2]],inplace=True)
seconddf.fillna('', inplace=True)
seconddf = seconddf.replace(',','', regex=True)
seconddf.drop(seconddf.index[0],inplace=True)
seconddf['Geography'] = seconddf['Geography'].str[4:]
seconddf['City'] = seconddf['Geography'].apply(lambda x: x.split(" ", 1)[0])
cols2=['Geography', 'City']
new_cols2= cols+(seconddf.columns.drop(cols2).tolist())
seconddf = seconddf[new_cols2]
seconddf.insert(2, 'State', statesdf['State'])

In [4]:
firstdf['International Migration'] = firstdf['International Migration'].astype(int)
firstdf['Domestic Migration'] = firstdf['Domestic Migration'].astype(int)
firstdf['International Migration']=firstdf['International Migration']/firstdf['International Migration'].max()
firstdf['Domestic Migration']=firstdf['Domestic Migration']/firstdf['Domestic Migration'].max()

In [5]:
seconddf['International Migration'] = seconddf['International Migration'].astype(int)
seconddf['Domestic Migration'] = seconddf['Domestic Migration'].astype(int)
seconddf['International Migration']=seconddf['International Migration']/seconddf['International Migration'].max()
seconddf['Domestic Migration']=seconddf['Domestic Migration']/seconddf['Domestic Migration'].max()

In [6]:
body = client_bb25f14d4df64666b23e2f24f875ea53.get_object(Bucket='capstoneproject-donotdelete-pr-tsoy8yandjxtj3',Key='US cities by population with geographic coordinates.csv')['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
df2 = pd.read_csv(body)
df2.drop((['Washington','Unnamed: 5','Unnamed: 6','Unnamed: 7','Alabama','AL']),axis=1,inplace=True)
df2.columns = ['City','State','Longitude','Latitiude']

In [7]:
firstdf2=pd.merge(firstdf,df2,on=['City','State'])
firstdf2.loc[~(firstdf2['Domestic Migration'] > 0), 'Domestic Migration']=np.nan
firstdf2 = firstdf2.replace(np.nan, 0)
seconddf2=pd.merge(seconddf,df2,on=['City','State'])
seconddf2.loc[~(seconddf2['Domestic Migration'] > 0), 'Domestic Migration']=np.nan
seconddf2 = seconddf2.replace(np.nan, 0)
firstdf2.head()

,Geography,City,State,Population Change 2010-2015,Natural Increase,Event:Births,Event:Deaths,Net Migration,International Migration,Domestic Migration,Longitude,Latitiude
0,Abilene TX Metro Area,Abilene,TX,4326,3744,12227,8483,531,0.000397,0.000000,32.448736,-99.733144
1,Akron OH Metro Area,Akron,OH,1036,3857,40031,36174,-1982,0.001423,0.000000,41.081445,-81.519005
2,Albany GA Metro Area,Albany,GA,-3974,4017,11412,7395,-7985,0.000123,0.000000,31.578507,-84.155741
3,Albany OR Metro Area,Albany,OR,3875,1452,7582,6130,2237,0.000026,0.003462,44.636511,-123.105928
4,Albuquerque NM Metro Area,Albuquerque,NM,20226,20566,56632,36066,-402,0.001321,0.000000,35.085334,-106.605553


In [9]:
!conda install -c anaconda xlrd --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - xlrd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    certifi-2020.6.20          |           py36_0         160 KB  anaconda
    ca-certificates-2020.6.24  |                0         133 KB  anaconda
    xlrd-1.2.0                 |             py_0         108 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following packages will be UPDATED:

    ca-certificates: 2020.6.24-0       --> 2020.6.24-0       anaconda
    certifi:         2020.6.20-py36_0  --> 2020.6.20-py36_0  anaconda
    openssl:         1.1.1g-h7b6447c_0 --> 1.1.1g-h7b6447c_0 anaconda
    xlrd:            1.2.0-py_0       

In [10]:
import folium
USmapfirstint = folium.Map(location=[37.6714,-98.8521], tiles='OpenStreetMap', zoom_start=4.5)
 
for i in range(0,len(firstdf2)):
   folium.Circle(
      location=[firstdf2.iloc[i]['Longitude'], firstdf2.iloc[i]['Latitiude']],
      popup=firstdf2.iloc[i]['Geography'],
      radius=firstdf2.iloc[i]['International Migration']*10000000,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(USmapfirstint)

 
USmapfirstint

In [11]:
USmapfirstdom = folium.Map(location=[37.6714,-98.8522], tiles='OpenStreetMap', zoom_start=4.5)

for i in range(0,len(firstdf2)):
   folium.Circle(
      location=[firstdf2.iloc[i]['Longitude'], firstdf2.iloc[i]['Latitiude']],
      popup=firstdf2.iloc[i]['Geography'],
      radius=firstdf2.iloc[i]['Domestic Migration']*1000000,
      color='cadetblue',
      fill=True,
      fill_color='cadetblue'
   ).add_to(USmapfirstdom)

 
USmapfirstdom

In [12]:
USmapsecondint = folium.Map(location=[37.6714,-98.8521], tiles='OpenStreetMap', zoom_start=4.5)
 
for i in range(0,len(firstdf2)):
   folium.Circle(
      location=[seconddf2.iloc[i]['Longitude'], seconddf2.iloc[i]['Latitiude']],
      popup=seconddf2.iloc[i]['Geography'],
      radius=seconddf2.iloc[i]['International Migration']*10000000,
      color='yellow',
      fill=True,
      fill_color='yellow'
   ).add_to(USmapsecondint)

 
USmapsecondint

In [13]:
USmapseconddom = folium.Map(location=[37.6714,-98.8522], tiles='OpenStreetMap', zoom_start=4.5)

for i in range(0,len(firstdf2)):
   folium.Circle(
      location=[seconddf2.iloc[i]['Longitude'], seconddf2.iloc[i]['Latitiude']],
      popup=seconddf2.iloc[i]['Geography'],
      radius=seconddf2.iloc[i]['Domestic Migration']*1000000,
      color='green',
      fill=True,
      fill_color='green'
   ).add_to(USmapseconddom)
 
USmapseconddom

In [14]:
#combining both domestic and international migration US from 2010-2015
import folium
USmap2010 = folium.Map(location=[37.6714,-98.8521], tiles='OpenStreetMap', zoom_start=4.5)
 
for i in range(0,len(firstdf2)):
   folium.Circle(
      location=[firstdf2.iloc[i]['Longitude'], firstdf2.iloc[i]['Latitiude']],
      popup=firstdf2.iloc[i]['Geography'],
      radius=firstdf2.iloc[i]['International Migration']*10000000,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(USmap2010)

for i in range(0,len(firstdf2)):
   folium.Circle(
      location=[firstdf2.iloc[i]['Longitude'], firstdf2.iloc[i]['Latitiude']],
      popup=firstdf2.iloc[i]['Geography'],
      radius=firstdf2.iloc[i]['Domestic Migration']*1000000,
      color='cadetblue',
      fill=True,
      fill_color='cadetblue'
   ).add_to(USmap2010)

USmap2010

In [86]:
USmap2014 = folium.Map(location=[37.6714,-98.8521], tiles='OpenStreetMap', zoom_start=4.5)
 
for i in range(0,len(firstdf2)):
   folium.Circle(
      location=[seconddf2.iloc[i]['Longitude'], seconddf2.iloc[i]['Latitiude']],
      popup=seconddf2.iloc[i]['Geography'],
      radius=seconddf2.iloc[i]['International Migration']*10000000,
      color='blue',
      fill=True,
      fill_color='blue'
   ).add_to(USmap2014)

for i in range(0,len(firstdf2)):
   folium.Circle(
      location=[seconddf2.iloc[i]['Longitude'], seconddf2.iloc[i]['Latitiude']],
      popup=seconddf2.iloc[i]['Geography'],
      radius=seconddf2.iloc[i]['Domestic Migration']*1000000,
      color='yellow',
      fill=True,
      fill_color='yellow'
   ).add_to(USmap2014)


USmap2014